In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sys
import random
from pprint import pprint

In [2]:
df = pd.read_csv('file2.csv')

In [3]:
samples = df.drop(columns = {'new_cases'})

In [4]:
labels = df['new_cases']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train_samples, valid_samples, train_labels, valid_labels = train_test_split(samples, labels, test_size=0.2)

In [7]:
train_samples, test_samples, train_labels, test_labels = train_test_split(train_samples, train_labels, test_size=0.2)

In [8]:
# train = pd.concat((train_samples,train_labels), axis = 1)
# test = pd.concat((test_samples, test_labels), axis = 1)
# valid = pd.concat((valid_samples, valid_labels), axis = 1)

In [9]:
# train.to_csv('train.csv',index=False)
# test.to_csv('test.csv',index=False)
# valid.to_csv('valid.csv',index=False)

In [10]:
# train = pd.read_csv('train.csv')

In [11]:
# columns = train.columns

In [12]:
from sklearn.preprocessing import StandardScaler
# from sklearn import preprocessing

In [13]:
# stand= StandardScaler()
# train = stand.fit_transform(train)
# train = pd.DataFrame(train, columns = columns)

In [14]:
# train = preprocessing.normalize(train)
# train = pd.DataFrame(train, columns = columns)

In [15]:
# train_samples = train.drop(columns = {'new_cases'})
# train_labels = train['new_cases']

In [31]:
train_labels.to_numpy().reshape(-1,1).shape

(267, 1)

In [16]:
scale = StandardScaler()
scaled_train_samples = scale.fit_transform(train_samples)
scaled_train_labels = scale.fit_transform(train_labels.to_numpy().reshape(-1,1))

In [17]:
train_samples.head()

,retail_and_recreation,residential,transit_stations,parks,workplaces,total_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,reproduction_rate,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,stringency_index
417,-60.143,23.286,-48.286,-38.000,-43.429,28175044.0,175167.143,331895.0,2795.0,3523.429,...,0.74,1683135.0,344866883.0,249.903,1.220,2047260.0,1.484,0.086,11.7,81.94
139,-50.857,13.286,-38.714,-52.714,-30.143,2767253.0,62516.429,52888.0,1091.0,971.000,...,1.08,899864.0,30941264.0,22.421,0.652,808488.0,0.586,0.077,12.9,85.65
56,-71.714,19.429,-48.143,-56.571,-39.857,158086.0,6579.714,4534.0,190.0,157.143,...,1.27,116041.0,3242160.0,2.349,0.084,104253.0,0.076,0.063,15.8,81.94
11,-76.857,28.000,-71.286,-50.714,-63.143,5916.0,559.714,178.0,28.0,17.143,...,1.91,13904.0,127919.0,0.093,0.010,11906.0,0.009,0.047,21.3,100.00
262,-28.143,11.714,-9.143,-17.571,-14.571,10075116.0,24135.857,146111.0,301.0,343.143,...,0.86,900134.0,162098329.0,117.462,0.652,1075906.0,0.780,0.022,44.6,68.98


In [18]:
# pip install tensorflow

In [19]:
from tensorflow.keras import models, layers
import tensorflow as tf

In [20]:
# Feature Extractor
def getFeatureRepresentation(x):
    tensors = tf.convert_to_tensor(x, dtype = float)
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(26,)))
    model.add(layers.Dense(60, activation='tanh'))
    model.add(layers.Dense(128, activation = 'tanh'))
    model.add(layers.Dense(60, activation = 'tanh'))
    model.compile()
    feature_representation = model.predict(x = tensors)
    return feature_representation

In [21]:
train_feature_repr = getFeatureRepresentation(scaled_train_samples)
# val_feature_repr = getFeatureRepresentation(valid_samples)
# test_feature_repr = getFeatureRepresentation(test_samples)

9/9 [==============================] - 0s 3ms/step


In [22]:
train_feature_repr.shape

(267, 60)

In [29]:
scaled_train_labels.size

267

In [27]:
#Task Label Generator
def taskLabelGenerator(feature_repr, labels):
    
    labels = np.reshape(labels, (labels.size, 1))
    print(labels.shape)
    feaRepLabel = np.concatenate((feature_repr, labels), axis = 1)
    tensors = tf.convert_to_tensor(feaRepLabel, dtype = float)
    
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(61,)))
    model.add(layers.Dense(128))
    model.add(layers.Dense(256))
    model.add(layers.Dense(128))
    model.add(layers.Dense(60))
    model.compile()
    task_labels = model.predict(x = tensors)
    task_label_mean = np.average(task_labels, axis = 0)
    print(task_label_mean)
    task_label = tf.keras.activations.sigmoid(task_label_mean)
    return task_label

In [28]:
train_task_label = taskLabelGenerator(train_feature_repr, scaled_train_labels)

(267, 1)
9/9 [==============================] - 0s 3ms/step
[-0.01992447  0.00496221 -0.03411863  0.01968615 -0.02552442 -0.03774665
 -0.00500588  0.04772207 -0.07418428 -0.02433455 -0.09111973  0.0239384
  0.06868155  0.00821153 -0.03178158 -0.0122079  -0.00541717 -0.02411699
  0.00184896 -0.0155035   0.04683812  0.04850907  0.00398424 -0.01815405
  0.03367265 -0.02831156 -0.02343228 -0.06194454  0.02375529 -0.05900317
  0.04306082  0.03436428 -0.05123675 -0.05312964  0.00736935  0.02023826
  0.02515413 -0.01663342 -0.0076221   0.05456482 -0.07881723  0.03946211
  0.01341752 -0.04080233 -0.01950396 -0.0249821   0.00436023 -0.05879458
 -0.00217525 -0.01592738  0.00233761  0.04206393  0.05521156 -0.01710947
  0.04669772  0.00851528 -0.00201486 -0.00181215 -0.005932   -0.04350811]


In [26]:
train_task_label

<tf.Tensor: shape=(60,), dtype=float32, numpy=
array([0.5132062 , 0.5190997 , 0.49908933, 0.50263536, 0.4997008 ,
       0.4952689 , 0.47461796, 0.50745904, 0.4860393 , 0.500226  ,
       0.4992364 , 0.48719874, 0.48992905, 0.50317234, 0.51177764,
       0.5211032 , 0.5204305 , 0.5077482 , 0.50832975, 0.48777062,
       0.5117642 , 0.49018914, 0.51840174, 0.5035096 , 0.4956466 ,
       0.47401887, 0.5077276 , 0.51462734, 0.4947219 , 0.5014126 ,
       0.5114862 , 0.5120846 , 0.5219123 , 0.5004596 , 0.50510406,
       0.5160289 , 0.4926571 , 0.508424  , 0.50553614, 0.48473194,
       0.5058138 , 0.48695564, 0.47535416, 0.49418333, 0.4986304 ,
       0.5106003 , 0.49410033, 0.5067023 , 0.49908113, 0.49115595,
       0.49814895, 0.48157182, 0.5121464 , 0.5097366 , 0.50813335,
       0.48171026, 0.4892957 , 0.5104917 , 0.49739113, 0.5014418 ],
      dtype=float32)>

In [2000]:
train_task_label.shape

TensorShape([60])

In [2001]:
#Weights Generator
def weightsGenerator(feature_repr, labels, task_label):
    
    labels = np.reshape(labels, (labels.size, 1))
    
    task_label = np.reshape(task_label, (1,60))
    task_label = np.tile(task_label, (labels.size, 1))
    
    feaRepLabelTaskLabel = np.concatenate((feature_repr, labels, task_label), axis = 1)
    tensor = tf.convert_to_tensor(feaRepLabelTaskLabel, dtype = float)
    
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(121,)))
    model.add(layers.Dense(128, activation = 'tanh'))
#     model.add(layers.Dense(420, activation = 'tanh'))
#     model.add(layers.Dense(526, activation = 'tanh'))
#     model.add(layers.Dense(1024, activation = 'tanh'))
#     model.add(layers.Dense(1024, activation = 'tanh')) 
#     model.add(layers.Dense(526, activation = 'tanh'))
#     model.add(layers.Dense(420, activation = 'tanh'))
    model.add(layers.Dense(128, activation = 'tanh'))
    model.add(layers.Dense(60, activation = 'tanh'))
    model.compile()
    weights = model.predict(x = tensor)
    weight_mean = np.average(weights, axis = 0)
    return weight_mean

In [2002]:
weights = weightsGenerator(train_feature_repr, scaled_train_labels, train_task_label)

9/9 [==============================] - 0s 2ms/step


In [2003]:
result = np.dot(train_feature_repr, weights)

In [2004]:
unscale_res = scale.inverse_transform(result.reshape(-1,1))

In [2005]:
unscale_labels = scale.inverse_transform(scaled_train_labels)

In [2006]:
from sklearn.metrics import mean_squared_error

In [2007]:
rms = mean_squared_error(unscale_labels, unscale_res, squared=False)
rms

104226.75294943272

Adding Self Attention Block

In [1862]:
pip install keras_self_attention

Note: you may need to restart the kernel to use updated packages.


In [1863]:
import keras
from keras_self_attention import SeqSelfAttention

In [1864]:
def selfAttentionBlock(model, embedding, residual_connection):
    model.add(SeqSelfAttention(attention_activation = 'sigmoid'))
    model.add(layers.Dense(128, activation = 'relu'))
    model.add(layers.Dense(64))
    output = model.add(layers.LayerNormalization(axis=1))
    model.layers.Add()([embedding, output])
    model.compile()
    output = model.predict(embedding)
    return output

In [1865]:
#Task Label Generator
def taskLabelGenerator(feature_repr, labels):
    
    labels = labels.to_numpy()
    labels = np.reshape(labels, (labels.size, 1))
    
    feaRepLabel = np.concatenate((feature_repr, labels), axis = 1)
    tensor = tf.convert_to_tensor(feaRepLabel, dtype = float)
    
    model = models.Sequential()
    
    #First Layer
    model.add(tf.keras.Input(shape=(41,)))
    
    #Second Layer
    embedding = model.add(layers.Dense(64))
    
    #First Self-Attention Block
    tensor = selfAttentionBlock(model, embedding, embedding)
    
    #Second Self-Attention Block
    tensor = selfAttentionBlock(model, tensor, embedding)
    
    #Third Self-Attention Block
    tensor = selfAttentionBlock(model, tensor, embedding)
    
    #Final Layer
    model.add(layer.Dense(40))
    
    model.compile()
    task_labels = model.predict(x = tensor)
    
    #Taking mean of all task labels
    task_label_mean = np.average(task_labels, axis = 0)
    
    #Applying Softmax
    layer = layers.Softmax()
    task_label = layer(task_label_mean).numpy()
    return task_label

In [1866]:
train_task_label = taskLabelGenerator(train_feature_repr, train_labels)

IndexError: tuple index out of range

In [ ]:
#Weights Generator
def weightsGenerator(feature_repr, labels, task_label):
    
    labels = labels.to_numpy()
    labels = np.reshape(labels, (labels.size, 1))
    
    task_label = np.reshape(task_label, (1,40))
    task_label = np.tile(task_label, (labels.size, 1))
    
    feaRepLabelTaskLabel = np.concatenate((feature_repr, labels, task_label), axis = 1)
    tensor = tf.convert_to_tensor(feaRepLabelTaskLabel, dtype = float)
    
    model = models.Sequential()
    
    #First Layer
    model.add(tf.keras.Input(shape=(81,)))
    
    #Second Layer
    embedding = model.add(layers.Dense(64))
    
    #First Self-Attention Block
    tensor = selfAttentionBlock(embedding, embedding)
    
    #Second Self-Attention Block
    tensor = selfAttentionBlock(tensor, embedding)
    
    #Third Self-Attention Block
    tensor = selfAttentionBlock(tensor, embedding)
    
    #Final Layer
    model.add(layers.Dense(40))
    
    model.compile()
    weights = model.predict(x = tensor)
    
    #Taking mean of all the weights
    weight_mean = np.average(weights, axis = 0)
    return weight_mean